# Leitura dos Dataset's

In [ ]:
import pandas as pd
import os

In [ ]:
df_list = []

# Caminho dos arquivos .csv
pathFiles = os.getcwd() + '\\data'

for file in os.listdir(pathFiles):
    filePath = os.path.join(pathFiles, file)
    df = pd.read_csv(filePath, sep=';', encoding='latin-1')
    df_list.append(df)
    #print(filePath)

df = pd.concat(df_list)
df.reset_index(drop=True, inplace=True)
df[:20]


# Conexão com MySQL

In [ ]:
import MySQLdb
import sqlalchemy
from sqlalchemy import text

Informações do servidor do banco de dados

In [ ]:
user = 'root'
password = 'mysqlpwd'
host = 'localhost'
database = 'coletaIGTI'

string_conexao = f'mysql://{user}:{password}@{host}/{database}'
string_conexao

Abre a conexão com o banco de dados

In [ ]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()

In [ ]:
# Dataframes de pessoas e modelos
df_pessoas = df[['cod_pessoa', 'nome', 'genero', 'data_nascimento']].drop_duplicates()
df_modelos = df[['modelo', 'fabricante']].drop_duplicates()

# Criado como uma lista pois possui apenas uma coluna
fabricantes = list(df.fabricante.unique())

Conferindo uma amostra dos dados

In [ ]:
df_pessoas[:10]

In [ ]:
df_modelos[:10]

In [ ]:
fabricantes

Insere os dados na tabela <i>pessoa</i>

In [ ]:
for pessoa in df_pessoas.itertuples():
    cod_pessoa = pessoa.cod_pessoa
    nome = pessoa.nome
    genero = pessoa.genero
    data_nascimento = str(pessoa.data_nascimento)
    try:
        query = f"""insert into pessoa (cod_pessoa, nome, genero, data_nascimento) values 
                    ('{cod_pessoa}', '{nome}', '{genero}', '{data_nascimento}')"""
        conn.execute(text(query))
        conn.commit()
        print(f'Registro inserido com sucesso: {nome}')

    except Exception as e:
        print(f'Não foi possível inserir o registro {nome}. O erro encontrado foi: {e}.')

In [ ]:
# Confirma toda a transação realizada no banco de dados
conn.commit()

Insere os dados na tabela <i>fabricante</i>

In [ ]:
for fabricante in fabricantes:
    try:
        query = f"insert into fabricante (nome_fabricante) values ('{fabricante}')"
        conn.execute(text(query))
        conn.commit()
        print(f'Registro inserido com sucesso: {fabricante}')

    except Exception as e:
        print(f'Não foi possível inserir o registro {fabricante}. O erro encontrado foi: {e}.')

In [ ]:
# Confirma toda a transação realizada no banco de dados
conn.commit()

In [ ]:
# Busca os fabricantes no banco de dados
df_fabricante = pd.read_sql('fabricante', con = conn)

In [ ]:
df_fabricante

Transforma o dataframe em uma collection dictionary para facilitar a busca para inserir os dados de modelos

In [ ]:
dict_fabricante = df_fabricante.set_index(df_fabricante.nome_fabricante)['cod_fabricante'].to_dict()
dict_fabricante

Realiza um map para associar o código do fabricante no dataframe de modelos

In [ ]:
df_modelos.fabricante = df_modelos.fabricante.map(dict_fabricante)
df_modelos

Feito o map, renomeia o nome da coluna

In [ ]:
df_modelos.rename(columns={'fabricante':'cod_fabricante'}, inplace=True)
df_modelos

Insere os dados na tabela <i>modelo_veiculo</i>

In [ ]:
for modelo in df_modelos.itertuples():
    nome_modelo = modelo.modelo
    cod_fabricante = modelo.cod_fabricante
    try:
        query = f"""insert into modelo_veiculo (descricao_modelo, cod_fabricante) values 
                    ('{nome_modelo}', {cod_fabricante})"""
        conn.execute(text(query))
        conn.commit()
        print(f'Registro inserido com sucesso: {nome_modelo}')

    except Exception as e:
        print(f'Não foi possível inserir o registro {nome_modelo}. O erro encontrado foi: {e}.')

In [ ]:
# Confirma toda a transação realizada no banco de dados
conn.commit()

# Criando um ambiente de Staging
Essa é uma outra forma de se trabalhar com os dados no banco de dados. Jogamos todo o dataframe original no banco de dados, e tratamos todas as informações dentro do próprio banco de dados, populando as tabelas necessárias

In [ ]:
df.to_sql('stg_venda_veiculo', con=conn, schema='coletaIGTI', if_exists='replace', index=False)

In [ ]:
# Confirma toda a transação realizada no banco de dados
conn.commit()

# !!! - Importante - !!!
Ao terminar todo o procedimento, não esqueça de fechar a conexão com o banco de dados, evitando problemas futuros de performance, memória e sobrecarregamento do banco. ;)

In [ ]:
# Fecha a conexão com o banco de dados
conn.close()